In [3]:
import pandas as pd
import os 

In [21]:
ruta_archivos = "D:/PROYECTO FINAL/Google Maps/metadata-sitios/"     # Ruta de los json de Sitios
archivos_json = [[os.path.join(ruta_archivos, archivo), archivo] for archivo in os.listdir(ruta_archivos) if archivo.endswith('.json')]

In [3]:
archivos_json

[['D:/PROYECTO FINAL/Google Maps/metadata-sitios/1.json', '1.json'],
 ['D:/PROYECTO FINAL/Google Maps/metadata-sitios/2.json', '2.json']]

In [4]:
df_estados_USA= pd.read_csv("D:/PROYECTO FINAL/Google Maps/estados_usa.csv",delimiter = ';', encoding = "utf-8")

In [5]:
df_sitios= pd.read_json('D:/PROYECTO FINAL/Google Maps/metadata-sitios/1.json', lines=True)

In [6]:
df_sitios= df_sitios.dropna(subset=['address']) # Elimina filas con address nulas
df_sitios= df_sitios.dropna(subset=['category']) # Elimina filas con category nulas

In [7]:
df_sitios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 262678 entries, 0 to 275000
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              262678 non-null  object 
 1   address           262678 non-null  object 
 2   gmap_id           262678 non-null  object 
 3   description       13063 non-null   object 
 4   latitude          262678 non-null  float64
 5   longitude         262678 non-null  float64
 6   category          262678 non-null  object 
 7   avg_rating        262678 non-null  float64
 8   num_of_reviews    262678 non-null  int64  
 9   price             13319 non-null   object 
 10  hours             183150 non-null  object 
 11  MISC              192649 non-null  object 
 12  state             186168 non-null  object 
 13  relative_results  229847 non-null  object 
 14  url               262678 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 32.1+ MB


In [8]:
df_sitios= df_sitios.explode('category')

In [9]:
df_sitios= df_sitios[df_sitios['category'] == "Restaurant"]

In [10]:
df_sitios= df_sitios.fillna({'price':'SIN DATO', 'state':'SIN DATO'})     # Se imputan los valores nulos a 'SIN DATO'

In [11]:
# Se guardan las columnas MISC y el gmap_id en un df
df_misc= df_sitios[['gmap_id','MISC']]
df_misc= df_misc.dropna(subset=['MISC']) # Elimina filas con MISC nulo

In [12]:
# Funcion para cargar los json a una serie de pandas
def expandir_diccionario(diccionario):
    return pd.Series(diccionario)

# Se concatenas las series al df original
df_expandido = pd.concat([df_misc, df_misc['MISC'].apply(expandir_diccionario)], axis=1)
df_expandido.drop('MISC', axis=1, inplace=True)     # Se elimina la columna de MISC
df_misc= df_expandido

In [13]:
df_Service_options= df_misc[['gmap_id','Service options']]      # Se crea un df solo las columnas gmap_id y Service Options
df_Service_options= df_Service_options.rename(columns={'Service options': 'service_option'}) # cambiar nombre de la columna
df_Service_options= df_Service_options.dropna(subset=['service_option']) # Elimina filas con columna 'Service Opciones nulas'
df_Service_options= df_Service_options.explode('service_option')

In [14]:
df_Service_options

,gmap_id,service_option
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Outdoor seating
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Curbside pickup
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,No-contact delivery
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Delivery
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Takeout
...,...,...
274958,0x89d6b577c1295421:0xb1f4ce1abf53d3b6,No-contact delivery
274958,0x89d6b577c1295421:0xb1f4ce1abf53d3b6,Delivery
274958,0x89d6b577c1295421:0xb1f4ce1abf53d3b6,Takeout
274958,0x89d6b577c1295421:0xb1f4ce1abf53d3b6,Dine-in


In [15]:
df_Planning= df_misc[['gmap_id','Planning']]      # Se crea un df solo las columnas gmap_id y Planning
df_Planning= df_Planning.rename(columns={'Planning': 'planning_option'}) # cambiar nombre de la columna
df_Planning= df_Planning.dropna(subset=['planning_option']) # Elimina filas con columna 'Service Opciones nulas'
df_Planning= df_Planning.explode('planning_option')

In [16]:
df_Planning

,gmap_id,planning_option
307,0x86275dc7cfc138d5:0x4c35fe08b16e7a2b,Accepts reservations
917,0x89c587762e7e69dd:0x30a60dce48b073c3,Accepts reservations
1165,0x88c2fea614c52c8b:0x50e166caa05e8c0f,Accepts reservations
1665,0x89e80e710fddbabb:0xdaa5f6da10c030b9,Accepts reservations
2147,0x89c25a275e5088dd:0xb6ecb4eaa82d1180,Quick visit
...,...,...
274086,0x89c2c70157450c41:0xd604da267031b8bc,Dinner reservations recommended
274086,0x89c2c70157450c41:0xd604da267031b8bc,Accepts reservations
274311,0x80c8daee5baf656d:0xd86ef2a8bfdb748b,Accepts reservations
274785,0x88f45b888accca4f:0xf5341d15da635764,Accepts reservations


In [17]:
# En pruebas se detecto que esta fila estaba dando problemas con la funcion para extraer, asi que se procede a eliminarla
df_sitios= df_sitios[df_sitios['address'] != "〒10028 New York, Lexington Ave, (New) Ichie Japanese Restaurant"]

In [18]:
def Ext_Ciudad_Estado(dir):
    ciudad= "SIN DATO"
    estado= "SIN DATO"     
    if len(str(dir)) > 10:  
        lista= str(dir).split(',')
        if len(lista) > 2:
            codigo= lista[-1][1:3]
            df_filtro= df_estados_USA[df_estados_USA['nombre_corto'].str.contains(codigo)]        
            if not df_filtro.empty:            
                ciudad = lista[-2].strip()
                estado= df_filtro.nombre_largo.values[0].strip()
    return ciudad, estado

In [19]:
df_sitios[['city','state']] = df_sitios.apply(lambda x: Ext_Ciudad_Estado(x['address']), axis=1, result_type='expand')

In [20]:
df_sitios

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,city
6,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,None,33.916402,-118.010855,Restaurant,4.5,18,SIN DATO,"[[Thursday, 11AM–9:30PM], [Friday, 11AM–9:30PM...","{'Service options': ['Outdoor seating', 'Curbs...",California,None,https://www.google.com/maps/place//data=!4m2!3...,La Mirada
68,Vivi Bubble Tea,"Vivi Bubble Tea, 701 S 5th St, Philadelphia, P...",0x89c6c89efcaed69d:0xded973f6033e7dba,"Bubble tea shop offering hot & cold versions, ...",39.940293,-75.150923,Restaurant,4.0,8,SIN DATO,None,"{'Service options': ['Delivery'], 'Amenities':...",Pennsylvania,"[0x89c6c62a40f4b733:0x73bfeec1898f272b, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...,Philadelphia
96,Hale Pops,"Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762",0x7c00456eecad3111:0x8217f9600c51f33,None,21.637796,-157.920714,Restaurant,4.4,18,SIN DATO,"[[Thursday, 11AM–8PM], [Friday, 11AM–8PM], [Sa...","{'Service options': ['Outdoor seating', 'Takeo...",Hawái,"[0x7c00451360f80cf1:0x930291a38bab3132, 0x7c00...",https://www.google.com/maps/place//data=!4m2!3...,Laie
114,Cape Seafood Shack,"Cape Seafood Shack, 603 Del Prado Blvd S, Cape...",0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,None,26.641377,-81.940545,Restaurant,5.0,1,$$,None,"{'Service options': ['Dine-in', 'Delivery'], '...",Florida,None,https://www.google.com/maps/place//data=!4m2!3...,Cape Coral
123,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",0x8644b59b8fe872e5:0x5e638876caa84cc3,None,30.273985,-97.719563,Restaurant,4.5,8,SIN DATO,"[[Thursday, 5PM–12AM], [Friday, 5PM–12AM], [Sa...","{'Service options': ['Delivery', 'Takeout', 'D...",Texas,None,https://www.google.com/maps/place//data=!4m2!3...,Austin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274875,Quaker Steak & Lube,"Quaker Steak & Lube, 3335 W Wood River Rd, Gra...",0x879984bc08a64061:0x7ab948f1bbff7c0e,Chicken wings & many spicy sauces star on the ...,40.830827,-98.380752,Restaurant,2.9,36,SIN DATO,None,"{'Service options': ['Delivery', 'Takeout'], '...",Nebraska,"[0x879984bc06565fb3:0x943988367cef2f83, 0x8799...",https://www.google.com/maps/place//data=!4m2!3...,Grand Island
274895,Subway,"Subway, 3657 E Main St, Whitehall, OH 43213",0x8838629ba011ece9:0x9b75b19079ad45e7,Casual counter-serve chain for build-your-own ...,39.954151,-82.900727,Restaurant,3.7,68,$,"[[Saturday, 8AM–8PM], [Sunday, 9AM–8PM], [Mond...","{'Service options': ['Delivery', 'Takeout', 'D...",Ohio,"[0x883863524d10f45b:0x77c4ee4aa707d85f, 0x8838...",https://www.google.com/maps/place//data=!4m2!3...,Whitehall
274896,ON THE RIVER SPORTS BAR AND GRILL,"ON THE RIVER SPORTS BAR AND GRILL, Gilbert, WV...",0x884fa3f049375197:0xf7ea720f164ef342,None,37.601291,-81.860280,Restaurant,4.4,18,SIN DATO,"[[Saturday, 11AM–12AM], [Sunday, 11AM–12AM], [...","{'Service options': ['Delivery'], 'Amenities':...",West Virginia,"[0x884f9f460c671809:0xe59570ca6f505c3c, 0x884f...",https://www.google.com/maps/place//data=!4m2!3...,Gilbert
274958,Palermo’s Market,"Palermo’s Market, Mercantile on Main, 240 E Ma...",0x89d6b577c1295421:0xb1f4ce1abf53d3b6,None,43.157746,-77.605956,Restaurant,4.9,76,SIN DATO,"[[Saturday, 8AM–7PM], [Sunday, Closed], [Monda...",{'From the business': ['Identifies as women-le...,New York,None,https://www.google.com/maps/place//data=!4m2!3...,Rochester


In [21]:
lista_estados= ['Florida', 'Pennsylvania', 'Tennessee', 'California', 'Texas', 'New York']
df_sitios= df_sitios[df_sitios['state'].isin(lista_estados)]

In [22]:
#Borrar Columnas
df_sitios= df_sitios.drop(['relative_results','address', 'num_of_reviews', 'description', 'url','category', 'MISC', 'hours'], axis=1) 
# Ordena el orden de las columnas
df_sitios= df_sitios[['gmap_id','name', 'city', 'state', 'latitude', 'longitude', 'avg_rating', 'price']]

In [23]:
df_sitios= df_sitios.rename(columns={'gmap_id': 'business_id', 'avg_rating': 'stars'}) # cambiar nombre de la columna

In [24]:
df_sitios['platform']= 1

In [25]:
df_sitios

,business_id,name,city,state,latitude,longitude,stars,price,platform
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,La Mirada,California,33.916402,-118.010855,4.5,SIN DATO,1
68,0x89c6c89efcaed69d:0xded973f6033e7dba,Vivi Bubble Tea,Philadelphia,Pennsylvania,39.940293,-75.150923,4.0,SIN DATO,1
114,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,Cape Seafood Shack,Cape Coral,Florida,26.641377,-81.940545,5.0,$$,1
123,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,Austin,Texas,30.273985,-97.719563,4.5,SIN DATO,1
271,0x808f87f90c1f661f:0xf384e804a61e0c0b,Mariscos el poblano,Oakland,California,37.764203,-122.214647,5.0,SIN DATO,1
...,...,...,...,...,...,...,...,...,...
274722,0x89e82d0591b06633:0x40e962ba688eb743,Deli boss & brunch,West Babylon,New York,40.719946,-73.355972,4.4,SIN DATO,1
274772,0x887f850c749005f9:0xed634f0483c645a4,Taqueria Los Compadres,Memphis,Tennessee,35.050547,-89.866261,4.9,SIN DATO,1
274781,0x89c2f404caa68913:0x592c8b89b6226a82,Pica Pollo 809,New York,New York,40.855985,-73.928615,4.1,SIN DATO,1
274958,0x89d6b577c1295421:0xb1f4ce1abf53d3b6,Palermo’s Market,Rochester,New York,43.157746,-77.605956,4.9,SIN DATO,1
